<h1 align ="center"> REST API OCR Enhanchment Samples</h1>
<hr>
   
# Chat Completions

In [1]:
import json
import os
import requests
import base64

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment_name, openai_api_base, openai_api_key and openai_api_version.

In [2]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
deployment_name = config_details['GPT-4V_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

In [3]:
image_file_path = "../../common/images/TravelAssistant.jpeg"  # Update with your image path
sys_message = "You are an AI assistant that helps people find information."
user_prompt = "Based on this flight information board, can you provide specifics for my trip to Zurich?"

# Encode the image in base64
with open(image_file_path, 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('ascii')

# Construct the API request URL
api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/extensions/chat/completions?api-version={openai_api_version}"

# Including the api-key in HTTP headers
headers = {
    "Content-Type": "application/json",
    "api-key": openai_api_key,
}

# Payload for the request
payload = {
  "model": "gpt-4-vision-preview",    
  "enhancements": {
    "ocr": {
      "enabled": True   # Enable OCR enhancement
    },
  },
  "messages": [
    {
      "role": "system",
      "content": [
          {
            "type": "text", 
            "text": sys_message
          }
      ]
    },
    {
      "role": "user",
      "content": [
          {
            "type": "text",
            "text": user_prompt # Prompt for the user
          },
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}" # Image to be processed
          }
          }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

# Send the request and handle the response
try:
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad HTTP status codes
    response_content = response.json()
    print(response_content['choices'][0]['message']['content'])  # Print the content of the response
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

Your flight to Zurich is LX 1101. It is scheduled to depart at 09:20. You should proceed to check-in for your flight. The departure is from Terminal 2. Please make sure to verify this information upon your arrival at the airport as schedules can change.
